In [13]:
import requests
from langchain.prompts import PromptTemplate
from langchain.llms.base import LLM
from google.oauth2 import service_account
import os
from dotenv import load_dotenv

In [5]:
# Google Gemini API configurations
API_URL = "https://gemini.googleapis.com/v1/projects/137941730896/locations/global/gemini"
API_KEY = os.getenv('API_KEY')

In [6]:
class GeminiLLM(LLM):
    def _call(self, prompt: str, stop=None) -> str:
        """
        This function sends a request to the Gemini API and gets a response.
        """
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {API_KEY}",
        }
        data = {
            "model": "gemini-1",  
            "input": prompt,
            "parameters": {"max_output_tokens": 500},
        }
        response = requests.post(API_URL, headers=headers, json=data)
        return response.json().get("output", "")

    @property
    def _identifying_params(self):
        return {"api_url": API_URL}

    @property
    def _llm_type(self):
        return "gemini"

In [7]:
# Define a Prompt Template
template = """
You are a healthcare assistant specialized in heart diseases. You will have a conversation with a user based on the following input:

User Input: {user_input}

Respond with heart disease information or advice tailored to their input.
"""

prompt = PromptTemplate(input_variables=["user_input"], template=template)

# Set up the Gemini API as the LLM (Assuming you've defined the GeminiLLM class)
llm = GeminiLLM()

# Create a LangChain pipeline using RunnableSequence
chain = prompt | llm  # This is now the new syntax

C:\Users\Admin\AppData\Local\Temp\ipykernel_10144\2500618890.py:16: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [ ]:
# Example user interaction
def get_response_from_chatbot(user_input):
    response = chain.run(user_input)
    return response


In [ ]:
# Example usage
user_input = "What are the symptoms of heart disease?"
response = get_response_from_chatbot(user_input)
print(response)